In [2]:
from dcs.models.np_classifier import NPClassifier

NPClassifier()

2024-12-30 17:40:41.731739: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-30 17:40:41.778017: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-30 17:40:41.779089: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 17:40:42.536068: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jcapela/miniforge3/envs/deepmol_models/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skippe

NPClassifier Overview

    This model is a reimplementation of NPClassifier described in 
https://pubs.acs.org/doi/10.1021/acs.jnatprod.1c00399. 

    All credits should be given to the authors. NPClassifier performs automated structural classification of 
natural products. 

    
                NPClassifier Details                 
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Attribute            ┃ Value                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Model Name           │ NPClassifier               │
│ Prediction Type      │ Pathway, Superclass, class │
└──────────────────────┴────────────────────────────┘
╭─────────────────────────── Key Features ───────────────────────────╮
│                                                                    │
│     - Prediction of natural product Pathway, Superclass and Class. │
│     - Efficient performance on large datasets.                     │
│                                                       

In [1]:
from dcs.models.np_classifier import NPClassifier

NPClassifier().predict_from_csv("test_predictions.csv", "smiles", "ID")

2024-12-31 19:19:39.500384: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-31 19:19:39.546143: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-31 19:19:39.547182: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-31 19:19:40.199232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jcapela/miniforge3/envs/deepmol_models/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skippe

Models already in cache.


/home/jcapela/miniforge3/envs/deepmol_models/lib/python3.9/site-packages/deepmol/compound_featurization/__init__.py:20: UserWarning: Mol2Vec not available. Please install it to use it. (pip install git+https://github.com/samoturk/mol2vec#egg=mol2vec)
  warnings.warn("Mol2Vec not available. Please install it to use it. "
2024-12-31 19:19:44.346284: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-12-31 19:19:45.022117: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://96214b804c9546dfacd101afd53419ea: INVALID_ARGUMENT: ram://96214b804c9546dfacd101afd53419ea is a directory.
2024-12-31 19:19:50.875512: W tensorflow/core/util/tensor_slice_reader.cc:97] 

2024-12-31 19:19:55,205 — ERROR — 'Functional' object has no attribute 'predict_proba'
2024-12-31 19:19:55,206 — INFO — <keras.src.engine.functional.Functional object at 0x7eea41c0a370>
2024-12-31 19:19:55,207 — INFO — <class 'keras.src.engine.functional.Functional'>


13/13 [==============================] - 0s 9ms/step


,ID,SMILES,Pathways,Superclass,Class
0,glucosamine,C(C1C(C(C(C(O1)O)N)O)O)O,Carbohydrates,Aminosugars and aminoglycosides,Aminosugars
1,n-nonane,CCCCCCCCC,Fatty acids,Fatty acyls,Hydrocarbons
2,plus quebrachitol,COC1C(C(C(C(C1O)O)O)O)O,Carbohydrates,Polyols,Cyclitols
3,galactosamine,C(C1C(C(C(C(O1)O)N)O)O)O,Carbohydrates,Aminosugars and aminoglycosides,Aminosugars
4,n-undecane,CCCCCCCCCCC,Fatty acids,Fatty acyls,Hydrocarbons
...,...,...,...,...,...
389,n-eicosane,CCCCCCCCCCCCCCCCCCCC,Fatty acids,Fatty acyls,Hydrocarbons
390,cannabielsoin acid b,CCCCCC1=CC(=C2C(=C1)OC(C(C2=O)CCC(=O)C)(C)C)O,"Polyketides, Terpenoids",Meroterpenoids,Cannabinoids
391,mannose,C(C1C(C(C(C(O1)O)O)O)O)O,Carbohydrates,Saccharides,Monosaccharides
392,plus inositol,C1(C(C(C(C(C1O)O)O)O)O)O,Fatty acids,Glycerophospholipids,"Cyclitols, Glycerophosphoinositols"


In [4]:
from dcs.models.plants_sm_predictor import PlantsSMPrecursorPredictor
PlantsSMPrecursorPredictor()

Plants secondary metabolite precursors predictor Overview

    This model is the same model available in https://github.com/jcapels/SMPrecursorPredictor/ 

    This model predicts plant secondary metabolite precursors. 
    
         Plants secondary metabolite precursors predictor Details          
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Attribute            ┃ Value                                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Model Name           │ Plants secondary metabolite precursors predictor │
│ Prediction Type      │ Precursor 1; Precursor 2                         │
└──────────────────────┴──────────────────────────────────────────────────┘
╭─────────────────────────────────────────── Key Features ────────────────────────────────────────────╮
│                                                                                                     │
│     - Prediction of plant seconda

In [2]:
from dcs.models import BBB,PPBR, VDss, Caco2, HIA, Bioavailability, \
    Lipophilicity, Solubility, CYP2D6Inhibition, CYP2C9Inhibition, \
        CYP3A4Inhibition, CYP2C9Substrate, CYP3A4Substrate, CYP2D6Substrate, HepatocyteClearance, MicrosomeClearance, hERGBlockers, LD50


results = Bioavailability().predict_from_csv("cannabinoids_small.csv", "smiles", "ID", output_file="predictions.csv")
results

2025-01-08 17:57:44.424256: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-08 17:57:44.480851: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-08 17:57:44.481884: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-08 17:57:45.422754: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jcapela/miniforge3/envs/deepmol_models/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skippe

Models already in cache.


DMPNNFeat: 100%|██████████| 72/72 [00:00<00:00, 376.46it/s]


,ID,SMILES,Bioavailability
0,cannabielsoic acid a,CCCCCC1=CC2=C(C3C(CCC(C3O2)(C)O)C(=C)C)C(=C1C(...,1.0
1,cannabielsoic acid b,CCCCCC1=CC(=C2C3C(CCC(C3OC2=C1C(=O)O)(C)O)C(=C...,1.0
2,cannabicyclol,CCCCCC1=CC(=C2C3C4C(C3(C)C)CCC4(OC2=C1)C)O,0.0
3,cannabicyclovarin,CCCC1=CC(=C2C3C4C(C3(C)C)CCC4(OC2=C1)C)O,1.0
4,cannabinolic acid a,CCCCCC1=CC2=C(C3=C(C=CC(=C3)C)C(O2)(C)C)C(=C1C...,0.0
...,...,...,...
67,ethylamine,CCN,1.0
68,piperidine,C1CCNCC1,0.0
69,dimethylamine,CNC,1.0
70,muscarine,CC1C(CC(O1)C[N+](C)(C)C)O,0.0


In [3]:
results = results.dropna()
results

,ID,SMILES,Bioavailability
0,cannabielsoic acid a,CCCCCC1=CC2=C(C3C(CCC(C3O2)(C)O)C(=C)C)C(=C1C(...,1.0
1,cannabielsoic acid b,CCCCCC1=CC(=C2C3C(CCC(C3OC2=C1C(=O)O)(C)O)C(=C...,1.0
2,cannabicyclol,CCCCCC1=CC(=C2C3C4C(C3(C)C)CCC4(OC2=C1)C)O,0.0
3,cannabicyclovarin,CCCC1=CC(=C2C3C4C(C3(C)C)CCC4(OC2=C1)C)O,1.0
4,cannabinolic acid a,CCCCCC1=CC2=C(C3=C(C=CC(=C3)C)C(O2)(C)C)C(=C1C...,0.0
...,...,...,...
67,ethylamine,CCN,1.0
68,piperidine,C1CCNCC1,0.0
69,dimethylamine,CNC,1.0
70,muscarine,CC1C(CC(O1)C[N+](C)(C)C)O,0.0


In [ ]:

from chemplot import Plotter, load_data

cp = Plotter.from_smiles(results["SMILES"], target=results["Bioavailability"], target_type="C", sim_type="structural")
cp.tsne()


figure(id='p1001', ...)

In [10]:
plot = cp.interactive_plot(show_plot=True)